In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neighbors import NearestNeighbors
from scipy import stats
import re
from tqdm import tqdm
import json
import sys


In [2]:
books = pd.read_csv('../../../data/recommendation/Books.csv',index_col=0)
ratings = pd.read_csv('../../../data/recommendation/Ratings.csv')

print(books.shape)
books.head()

C:\Users\PAC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(271360, 7)


,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
ISBN,,,,,,,
0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [3]:
def weighted_rating(v, R, C, m=5):
    w_r = (v/(v+m))*R + (m/(m+v))*C
    return w_r

R = ratings[['ISBN', 'Book-Rating']].groupby(['ISBN'], as_index=True).mean() # Book wise average rating
C = ratings['Book-Rating'].mean() # Average rating for all books

In [4]:
%%time
m=5
ratings = pd.concat([R, ratings.groupby(['ISBN'], as_index=True).size().to_frame()],axis=1).rename(columns={'Book-Rating':'average_rating',0:'votes'})
ratings['weighted_average'] = ratings.apply(lambda row : weighted_rating(row['votes'], row['average_rating'], C), axis = 1)

ratings.head()

Wall time: 12.6 s


,average_rating,votes,weighted_average
ISBN,,,
0330299891,3.0,2,2.904964
0375404120,1.5,2,2.476393
0586045007,0.0,1,2.389125
9022906116,3.5,2,3.047822
9032803328,0.0,1,2.389125


In [5]:
index = [idx for idx in books.index if idx in ratings.index]
df = pd.concat([books.loc[index,:], ratings.loc[index, :]], axis=1)
df = df[~df.index.duplicated(keep='first')]

df.head()

,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,average_rating,votes,weighted_average
ISBN,,,,,,,,,,
0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,0.000000,1,2.389125
0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,4.928571,14,4.386040
0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,5.000000,3,3.666844
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,4.272727,11,3.833422
0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,0.000000,1,2.389125


In [ ]:
# Action and Adventure
# Classics
# Comic Book or Graphic Novel
# Detective and Mystery
# Fantasy
# Historical Fiction
# Horror
# Literary Fiction
# Romance
# Science Fiction (Sci-Fi)
# Short Stories
# Suspense and Thrillers
# Women's Fiction
# Biographies and Autobiographies
# Cookbooks
# Essays
# History
# Memoir
# Poetry
# Self-Help
# True Crime